# Lab5 by Boying Li(boylil.3)

In [ ]:
import numpy as np
import sys

In [ ]:
#functions of non-linear activations
def f_sigmoid(X, deriv=False):
    if not deriv:
        return 1 / (1 + np.exp(-X))
    else:
        return f_sigmoid(X)*(1 - f_sigmoid(X))


def f_softmax(X):
    Z = np.sum(np.exp(X), axis=1)
    Z = Z.reshape(Z.shape[0], 1)
    return np.exp(X) / Z


In [ ]:
def exit_with_err(err_str):
    print(err_str, file=sys.stderr)
    sys.exit(1)

In [ ]:
#Functionality of a single hidden layer
class Layer:
    def __init__(self, size, batch_size, is_input=False, is_output=False,
                 activation=f_sigmoid):
        self.is_input = is_input
        self.is_output = is_output

        # Z is the matrix that holds output values
        self.Z = np.zeros((batch_size, size[0]))
        # The activation function is an externally defined function (with a
        # derivative) that is stored here
        self.activation = activation

        # W is the outgoing weight matrix for this layer
        self.W = None
        # S is the matrix that holds the inputs to this layer
        self.S = None
        # D is the matrix that holds the deltas for this layer
        self.D = None
        # Fp is the matrix that holds the derivatives of the activation function
        self.Fp = None

        if not is_input:
            self.S = np.zeros((batch_size, size[0]))
            self.D = np.zeros((batch_size, size[0]))

        if not is_output:
            self.W = np.random.normal(size=size, scale=1E-4)

        if not is_input and not is_output:
            self.Fp = np.zeros((size[0], batch_size))

    def forward_propagate(self):
        if self.is_input:
            return self.Z.dot(self.W)

        self.Z = self.activation(self.S)
        if self.is_output:
            return self.Z
        else:
            # For hidden layers, we add the bias values here
            self.Z = np.append(self.Z, np.ones((self.Z.shape[0], 1)), axis=1)
            self.Fp = self.activation(self.S, deriv=True).T
            return self.Z.dot(self.W)


In [ ]:
class MultiLayerPerceptron:
    def __init__(self, layer_config, batch_size=100):
        self.layers = []
        self.num_layers = len(layer_config)
        self.minibatch_size = batch_size

        for i in range(self.num_layers-1):
            if i == 0:
                print ("Initializing input layer with size {0}.".format(layer_config[i]))
                # Here, we add an additional unit at the input for the bias
                # weight.
                self.layers.append(Layer([layer_config[i]+1, layer_config[i+1]],
                                         batch_size,
                                         is_input=True))
            else:
                print ("Initializing hidden layer with size {0}.".format(layer_config[i]))
                # Here we add an additional unit in the hidden layers for the
                # bias weight.
                self.layers.append(Layer([layer_config[i]+1, layer_config[i+1]],
                                         batch_size,
                                         activation=f_sigmoid))

        print ("Initializing output layer with size {0}.".format(layer_config[-1]))
        self.layers.append(Layer([layer_config[-1], None],
                                 batch_size,
                                 is_output=True,
                                 activation=f_softmax))
        print ("Done!")

    def forward_propagate(self, data):
        # We need to be sure to add bias values to the input
        self.layers[0].Z = np.append(data, np.ones((data.shape[0], 1)), axis=1)

        for i in range(self.num_layers-1):
            self.layers[i+1].S = self.layers[i].forward_propagate()
        return self.layers[-1].forward_propagate()

    def backpropagate(self, yhat, labels):

        #-----------------------------------------------------------
        # exit_with_err("FIND ME IN THE CODE, What is computed in the next line of code?\n")
        # compute the output layer error
        #-----------------------------------------------------------

        self.layers[-1].D = (yhat - labels).T
        for i in range(self.num_layers-2, 0, -1):
            # We do not calculate deltas for the bias values
            W_nobias = self.layers[i].W[0:-1, :]
            #------------------------------------------------------------
            #exit_with_err("FIND ME IN THE CODE, What does this 'for' loop do?\n")
            #backpropagate the error layer by layer
            #---------------------------------------------------------

            self.layers[i].D = W_nobias.dot(self.layers[i+1].D) * self.layers[i].Fp

    def update_weights(self, eta):
        for i in range(0, self.num_layers-1):
            W_grad = -eta*(self.layers[i+1].D.dot(self.layers[i].Z)).T
            self.layers[i].W += W_grad

    def evaluate(self, train_data, train_labels, test_data, test_labels,
                 num_epochs=70, eta=0.05, eval_train=False, eval_test=True):

        N_train = len(train_labels)*len(train_labels[0])
        N_test = len(test_labels)*len(test_labels[0])

        print ("Training for {0} epochs...".format(num_epochs))
        for t in range(0, num_epochs):
            out_str = "[{0:4d}] ".format(t)

            for b_data, b_labels in zip(train_data, train_labels):
                output = self.forward_propagate(b_data)
                self.backpropagate(output, b_labels)
                #-----------------------------------------------
                #exit_with_err("FIND ME IN THE CODE, How does weight update is implemented? What is eta?\n")
                # eta is learning rate, in function update_weights() the gradient of weight is computed by
                # the dot product of the delta(error) and the current output
                #------------------------------------------------

                self.update_weights(eta=eta)

            if eval_train:
                errs = 0
                for b_data, b_labels in zip(train_data, train_labels):
                    output = self.forward_propagate(b_data)
                    yhat = np.argmax(output, axis=1)
                    errs += np.sum(1-b_labels[np.arange(len(b_labels)), yhat])

                out_str = ("{0} Training error: {1:.5f}".format(out_str,
                                                           float(errs)/N_train))

            if eval_test:
                errs = 0
                for b_data, b_labels in zip(test_data, test_labels):
                    output = self.forward_propagate(b_data)
                    yhat = np.argmax(output, axis=1)

                    #-----------add accuracy---------------------
                    acc = np.sum(yhat == np.argmax(b_labels,axis=1))/len(b_labels)
                    #--------------------------------------------
                    errs += np.sum(1-b_labels[np.arange(len(b_labels)), yhat])

                out_str = ("{0} Test error: {1:.5f}").format(out_str,
                                                       float(errs)/N_test) + f" Test accuracy: {acc}"


            print(out_str)


In [ ]:
def label_to_bit_vector(labels, nbits):
    bit_vector = np.zeros((labels.shape[0], nbits))
    for i in range(labels.shape[0]):
        bit_vector[i, labels[i]] = 1.0

    return bit_vector

In [ ]:
def create_batches(data, labels, batch_size, create_bit_vector=False):
    N = data.shape[0]
    print ("Batch size {0}, the number of examples {1}.".format(batch_size,N))

    if N % batch_size != 0:
        print ("Warning in create_minibatches(): Batch size {0} does not " \
              "evenly divide the number of examples {1}.".format(batch_size,N))
    chunked_data = []
    chunked_labels = []
    idx = 0
    while idx + batch_size <= N:
        chunked_data.append(data[idx:idx+batch_size, :])
        if not create_bit_vector:
            chunked_labels.append(labels[idx:idx+batch_size])
        else:
            bit_vector = label_to_bit_vector(labels[idx:idx+batch_size], 10)
            chunked_labels.append(bit_vector)

        idx += batch_size

    return chunked_data, chunked_labels


In [ ]:
def prepare_for_backprop(batch_size, Train_images, Train_labels, Valid_images, Valid_labels):

    print ("Creating data...")
    batched_train_data, batched_train_labels = create_batches(Train_images, Train_labels,
                                              batch_size,
                                              create_bit_vector=True)
    batched_valid_data, batched_valid_labels = create_batches(Valid_images, Valid_labels,
                                              batch_size,
                                              create_bit_vector=True)
    print ("Done!")


    return batched_train_data, batched_train_labels,  batched_valid_data, batched_valid_labels



In [ ]:
from keras.datasets import mnist

In [ ]:
(Xtr, Ltr), (X_test, L_test)=mnist.load_data()

Xtr = Xtr.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
Xtr = Xtr.astype('float32')
X_test = X_test.astype('float32')
Xtr /= 255
X_test /= 255
print(Xtr.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
60000 train samples
10000 test samples


# Task 1.1
a. the idea of backprop is that we get the loss/how wrong the output compared with the ground truth from the output layer(first forward pass), and we get how each layer contribute to the loss backwards, using chain rule to compute the gradient of loss with respect to weights.

b. softmax() is commonly used in multiclass classification problem, it outputs the probabilities, so in the code block above we use np.argmax() to find the class that has the max probability as the predicted class.

c. sigmoid (vanishing gradient problem, useful in binary classification output), tanh(vanishing gradient problem), relu (dying relu problem) and the variant leaky relu, softmax(useful in multiple classification)

d. (1) compute the output layer error (2) the loop is backpropagate the error layer by layer (3) eta is learning rate, in function update_weights() the gradient of weight is computed by the dot product of the delta(error) and the current output. What I am not sure the delta calcualtion (yhat - labels), is it coming from MSE's deriative and ignore the magnitude 2???



# Task 1.2 epochs = 70 and learning rate =0.05

In [ ]:
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels,
             eval_train=True)

print("Done:)\n")


Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.48887 Test error: 0.49390 Test accuracy: 0.6
[   1]  Training error: 0.10420 Test error: 0.10500 Test accuracy: 0.92
[   2]  Training error: 0.08580 Test error: 0.08670 Test accuracy: 0.91
[   3]  Training error: 0.04715 Test error: 0.05230 Test accuracy: 0.92
[   4]  Training error: 0.03990 Test error: 0.04480 Test accuracy: 0.96
[   5]  Training error: 0.03695 Test error: 0.04600 Test accuracy: 0.97
[   6]  Training error: 0.02850 Test error: 0.03740 Test accuracy: 0.98
[   7]  Training error: 0.02552 Test error: 0.03520 Test accuracy: 0.98
[   8]  Training error: 0.02415 Test error: 0.03750 Test accuracy: 0.97
[   9]  Training error: 0.02252 Test error: 

Note: We can see the overfitting quite soon as the test loss is going flat while the training loss is still decreasing and less than the test loss

# Task 1.3 Learning rate =0.5 and 0.005

In [ ]:
# eta = 0.5
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels, eta=0.5,
             eval_train=True)

print("Done:)\n")


Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.89782 Test error: 0.89900 Test accuracy: 0.1
[   1]  Training error: 0.88763 Test error: 0.88650 Test accuracy: 0.12
[   2]  Training error: 0.90070 Test error: 0.89680 Test accuracy: 0.09
[   3]  Training error: 0.90248 Test error: 0.90260 Test accuracy: 0.1
[   4]  Training error: 0.90248 Test error: 0.90260 Test accuracy: 0.1
[   5]  Training error: 0.90248 Test error: 0.90260 Test accuracy: 0.1
[   6]  Training error: 0.90263 Test error: 0.90180 Test accuracy: 0.12
[   7]  Training error: 0.90263 Test error: 0.90180 Test accuracy: 0.12
[   8]  Training error: 0.90137 Test error: 0.90420 Test accuracy: 0.12
[   9]  Training error: 0.90263 Test error: 0.9

In [ ]:
# eta = 0.005
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels, eta=0.005,
             eval_train=True)

print("Done:)\n")


Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.70337 Test error: 0.70100 Test accuracy: 0.3
[   1]  Training error: 0.64720 Test error: 0.64300 Test accuracy: 0.4
[   2]  Training error: 0.59818 Test error: 0.59680 Test accuracy: 0.43
[   3]  Training error: 0.45548 Test error: 0.46660 Test accuracy: 0.56
[   4]  Training error: 0.20697 Test error: 0.19770 Test accuracy: 0.81
[   5]  Training error: 0.11350 Test error: 0.11090 Test accuracy: 0.88
[   6]  Training error: 0.08995 Test error: 0.08750 Test accuracy: 0.87
[   7]  Training error: 0.07548 Test error: 0.07410 Test accuracy: 0.89
[   8]  Training error: 0.06405 Test error: 0.06400 Test accuracy: 0.93
[   9]  Training error: 0.05452 Test error: 0

Summary of learning rate: compared to 0.05, with 0.005 the overfitting is coming in later epochs. And with 0.5 learning rate, it is clearly too big so both the train loss and test loss remains high and fluctating and not able to find the optimal point.

# Task 1.4 Extend with Relu output

In [ ]:
#functions of non-linear activations
def f_relu(X, deriv=False, alpha=0.01):
    if not deriv:
      return np.maximum(X, 0)

    else:
      return np.where(X > 0, 1, X*alpha) # leaky relu


In [ ]:
class MLPReLu(MultiLayerPerceptron):
    def __init__(self, layer_config, batch_size=100):
        self.layers = []
        self.num_layers = len(layer_config)
        self.minibatch_size = batch_size

        for i in range(self.num_layers-1):
            if i == 0:
                print ("Initializing input layer with size {0}.".format(layer_config[i]))
                # Here, we add an additional unit at the input for the bias
                # weight.
                self.layers.append(Layer([layer_config[i]+1, layer_config[i+1]],
                                         batch_size,
                                         is_input=True,
                                         activation=f_relu))
            else:
                print ("Initializing hidden layer with size {0}.".format(layer_config[i]))
                # Here we add an additional unit in the hidden layers for the
                # bias weight.
                self.layers.append(Layer([layer_config[i]+1, layer_config[i+1]],
                                         batch_size,
                                         activation=f_sigmoid))

        print ("Initializing output layer with size {0}.".format(layer_config[-1]))
        self.layers.append(Layer([layer_config[-1], None],
                                 batch_size,
                                 is_output=True,
                                 activation=f_softmax))
        print ("Done!")

In [ ]:
# epochs = 70 and learning rate =0.05 with relu
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MLPReLu(layer_config=[784, 100, 100, 10], batch_size=batch_size)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels,
             eval_train=True)

print("Done:)\n")

Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.42535 Test error: 0.42240 Test accuracy: 0.63
[   1]  Training error: 0.09563 Test error: 0.09470 Test accuracy: 0.87
[   2]  Training error: 0.05807 Test error: 0.06080 Test accuracy: 0.91
[   3]  Training error: 0.04642 Test error: 0.04900 Test accuracy: 0.95
[   4]  Training error: 0.03235 Test error: 0.04110 Test accuracy: 0.96
[   5]  Training error: 0.03347 Test error: 0.04220 Test accuracy: 0.95
[   6]  Training error: 0.02710 Test error: 0.03880 Test accuracy: 0.96
[   7]  Training error: 0.02708 Test error: 0.03930 Test accuracy: 0.96
[   8]  Training error: 0.02598 Test error: 0.04000 Test accuracy: 0.97
[   9]  Training error: 0.02315 Test error:

Note: I have tried learning rate range from 0.005 to 1 for relu activation(the code blocks are removed to avoid messing up), but I failed to find a proper learning rate that get the training started and not too large so the loss is fluctuating. So instead of normal relu, I actually implemented the leaky relu which can help with dying relu problem. If using the normal relu, I think implementing some weight initialize techniques can help dying relu problem.